In [1]:
import pandas as pd
import numpy as np
import matminer
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_validate
from matminer.figrecipes.plot import PlotlyFig

In [2]:
data = pd.read_csv('ML_capacity.csv')

In [101]:
df_Li['structure']

0       Full Formula (P4 W8 O32)\r\nReduced Formula: P...
1       Full Formula (P12 W8 O52)\r\nReduced Formula: ...
2       Full Formula (P16 W6 O58)\r\nReduced Formula: ...
3       Full Formula (P4 W2 O14)\r\nReduced Formula: P...
4       Full Formula (P4 W2 O14)\r\nReduced Formula: P...
                              ...                        
2590    Full Formula (Cu2 P2 O8 F2)\r\nReduced Formula...
2602    Full Formula (Li1 Sb1 Te3 O12)\r\nReduced Form...
2603    Full Formula (Li1 Sb1 Te3 O12)\r\nReduced Form...
2604    Full Formula (Na12 Sb4 P8 O36)\r\nReduced Form...
2860    Full Formula (Li2 Zr7 Fe1 P12 O48)\r\nReduced ...
Name: structure, Length: 2291, dtype: object

In [3]:
df_Li = data[data['working_ion']=='Li']

In [4]:
mean = df_Li['average_voltage'].mean()
std = df_Li['average_voltage'].std()
up = mean+2*std
dw= mean-2*std

In [5]:
df1=df_Li[df_Li['average_voltage']< up]
df = df1[df1['average_voltage']> dw]

In [6]:
y = df ['average_voltage'].values
excluded = ['Unnamed: 0','formula_charge', 'formula_discharge', 'structure',
            'average_voltage', 'working_ion', 'compound possible','volume_change','energy_vol','energy_grav',
           'capacity_grav','capacity_vol','discharge_frac','charge_frac','max_delta_volume']
X = df.drop(excluded, axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.25, random_state=10)
scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform (X_test)

In [8]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate
svr = SVR (kernel = 'rbf', gamma =0.1, C = 10, epsilon= 0.1,degree=3)
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scores = cross_validate(svr, X_train_transformed, y_train, cv=cv, scoring = ('r2', 'neg_mean_absolute_error',
                                                                             'neg_mean_squared_error'))

In [9]:
svr.fit(X_train_transformed, y_train)
score = svr.score(X_test_transformed, y_test)
print('test R2 = ' + str(round(svr.score(X_test_transformed, y_test), 3)))
print('test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, y_pred=svr.predict(X_test_transformed))))
print('test MAE = %.3f' % mean_absolute_error(y_true=y_test, y_pred=svr.predict(X_test_transformed)))

test R2 = 0.6
test RMSE = 0.551
test MAE = 0.387


In [90]:
from sklearn.kernel_ridge import KernelRidge
KRR = KernelRidge (kernel = 'rbf',gamma = 0.1,alpha = 0.1, degree=3,kernel_params=None)

In [91]:
%%time
cv = ShuffleSplit(n_splits=10, test_size=0.25, random_state=10)
scores = cross_validate(KRR, X_train_transformed, y_train, cv=cv, scoring =('r2', 
                                                                            'neg_mean_absolute_error', 
                                                                            'neg_mean_squared_error'))

Wall time: 804 ms


In [92]:
KRR.fit (X_train_transformed, y_train)
#KRR.score (X_test_transformed, y_test)
print('test R2 = ' + str(round(KRR.score(X_test_transformed, y_test), 3)))
print('test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, 
                                                          y_pred=KRR.predict(X_test_transformed))))
print('test MAE = %.3f' % mean_absolute_error(y_true=y_test, y_pred=KRR.predict(X_test_transformed)))

test R2 = 0.6
test RMSE = 0.551
test MAE = 0.396


In [93]:
y_pred=svr.predict(X_test_transformed)

In [94]:
y_pred

array([3.00216925, 4.22832212, 3.23450973, 4.13095673, 3.13325432,
       3.96642666, 5.20232407, 3.58663043, 1.74820825, 1.97639904,
       3.90544096, 2.26862755, 3.84015172, 4.2061336 , 4.55469796,
       3.45473643, 4.2372872 , 2.25105214, 4.01686706, 2.05649387,
       2.58846315, 4.30104724, 4.49347818, 4.43372955, 5.12626137,
       4.38100386, 3.67280207, 3.86494014, 2.65699087, 4.23037064,
       4.35300064, 2.75758042, 3.21361269, 3.98884329, 2.92957079,
       3.41749121, 3.86334297, 3.03183001, 4.10621634, 4.82530757,
       2.06826552, 3.80403406, 4.56952857, 3.44965361, 4.05006591,
       4.77634291, 3.91043458, 4.4160026 , 4.51844285, 3.76446333,
       3.80665462, 2.02379752, 4.62281602, 3.58973768, 3.83761363,
       2.93042938, 3.11189988, 3.61834502, 2.45503484, 3.70402051,
       3.42867093, 3.93810725, 1.81480177, 3.86534855, 2.63815031,
       3.41208319, 3.87344366, 3.74954582, 4.09547668, 3.55793941,
       3.80989652, 3.43737955, 3.35030122, 3.12758025, 3.67975

# getting prediction data

In [10]:
#%matplotlib inline
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital, IonProperty
from matminer.featurizers.structure import (SiteStatsFingerprint, StructuralHeterogeneity,
                                            ChemicalOrdering, StructureComposition, MaximumPackingEfficiency)

C:\zxy\software\anacoda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.unsupervised module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [11]:
from matminer.featurizers.conversions import DictToObject

In [12]:
import numpy as np
import json

In [13]:
import pandas as pd
import pymatgen as mg
import csv

In [14]:
featurizer = MultipleFeaturizer([
    SiteStatsFingerprint.from_preset("CoordinationNumber_ward-prb-2017"),
    StructuralHeterogeneity(),
    ChemicalOrdering(),
    MaximumPackingEfficiency(),
    SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017"),
    StructureComposition(Stoichiometry()),
    StructureComposition(ElementProperty.from_preset("magpie")),
    StructureComposition(ValenceOrbital(props=['frac'])),
    StructureComposition(IonProperty(fast=True))
])

In [15]:
df_Al=pd.DataFrame(columns=['structure', 'file_name'])

In [16]:
import os
from pymatgen.core import Structure

In [19]:
folder_path=os.path.abspath('C:/Users/zhang/workshop-2017/cgcnn-master/my_cgcnn/CGCNN_web/web_voltage/prediction/Al_pred/root_vol_pred_Al/')

In [20]:
for i,j,k in os.walk(folder_path):
    filename_list=k

In [30]:
k[150]

'mvc-9418.cif'

In [31]:
for filename in filename_list[2:151]:
    filename_s=os.path.join(folder_path, filename)
    s=Structure.from_file(filename_s)
    df_Al=df_Al.append({'structure':s, 'file_name':filename }, ignore_index=True)

C:\zxy\software\anacoda3\lib\site-packages\pymatgen\io\cif.py:1116: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [47]:
voltage_file=os.path.join(folder_path, 'id_prop.csv')
al_V=pd.read_csv(voltage_file)

In [49]:
df_Al

,structure,file_name,spcacegroup_number,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean CN_VoronoiNN,avg_dev CN_VoronoiNN,mean absolute deviation in relative bond length,max relative bond length,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,compound possible,max ionic char,avg ionic char
0,"[[0. 0. 0.] Al, [1.6 1.6 1.6] Cu]",mp-1022721.cif,221,11.956922,11.956922,1.776357e-15,11.956922,8.881784e-16,0.000000,1.000000,...,225.000000,0.000000,225.0,0.214286,0.071429,0.714286,0.000000,False,0.020806,0.005201
1,"[[0.91759484 4.49434244 8.04969967] Sr, [-0.68...",mp-1041474.cif,46,5.276693,9.964681,4.687988e+00,8.479494,1.262424e+00,0.093431,1.151553,...,96.833333,98.972222,12.0,0.380952,0.444444,0.174603,0.000000,False,0.787757,0.171452
2,"[[4.00319498 2.27224666 2.40977645] Ti, [1.334...",mp-1041486.cif,13,5.876335,12.841077,6.964743e+00,10.411895,2.452624e+00,0.110333,1.116329,...,68.000000,81.454545,12.0,0.239130,0.347826,0.108696,0.304348,True,0.594445,0.075305
3,"[[4.03936968 2.14745679 2.56291005] Mo, [1.346...",mp-1041534.cif,13,5.844004,12.841534,6.997530e+00,10.340822,2.402959e+00,0.109013,1.114006,...,71.181818,86.082645,12.0,0.223404,0.340426,0.138298,0.297872,True,0.336084,0.055831
4,"[[0.98697611 4.52965491 7.4374028 ] Sr, [-0.70...",mp-1041550.cif,46,5.739410,9.779969,4.040559e+00,8.625092,1.039581e+00,0.083688,1.124143,...,98.166667,100.527778,12.0,0.347826,0.405797,0.246377,0.000000,False,0.787757,0.161617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,"[[-1.74107977 7.03647892 2.46224834] Ti, [ 5...",mvc-4059.cif,230,4.791369,9.457338,4.665969e+00,8.436354,1.361205e+00,0.083187,1.062393,...,78.166667,88.222222,12.0,0.375000,0.562500,0.062500,0.000000,True,0.594445,0.116633
145,"[[1.74759748 1.00897581 7.4144279 ] Si, [ 6.99...",mvc-4354.cif,230,4.768843,9.478858,4.710015e+00,8.447028,1.375650e+00,0.083646,1.062737,...,84.000000,96.000000,12.0,0.250000,0.375000,0.083333,0.291667,True,0.447278,0.079232
146,"[[1.73998689 1.00458183 7.38213889] Si, [ 6.95...",mvc-4519.cif,230,4.796627,9.442975,4.646348e+00,8.427611,1.353711e+00,0.082631,1.061975,...,83.333333,95.111111,12.0,0.315789,0.473684,0.210526,0.000000,False,0.447278,0.098923
147,"[[ 0.22280016 1.27259176 -0.23006298] Re, [3....",mvc-637.cif,2,5.578156,11.752605,6.174449e+00,9.722297,2.000784e+00,0.114281,1.130567,...,68.000000,81.454545,12.0,0.201835,0.293578,0.119266,0.385321,True,0.447278,0.063869


In [53]:
df_Al[df_Al.file_name=='mvc-9418.cif'].index.to_list()[0]

148

In [54]:
df_Al['voltage']=''
for index, rows in al_V.iterrows():
    ids=al_V['entry_id'][index]
    voltage=al_V['ave_voltage'][index]
    names=ids+'.cif'
    al_index=df_Al[df_Al.file_name==names].index.to_list()[0]
    df_Al['voltage'][al_index]=voltage

In [55]:
df_Al

,structure,file_name,spcacegroup_number,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean CN_VoronoiNN,avg_dev CN_VoronoiNN,mean absolute deviation in relative bond length,max relative bond length,...,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,compound possible,max ionic char,avg ionic char,voltage
0,"[[0. 0. 0.] Al, [1.6 1.6 1.6] Cu]",mp-1022721.cif,221,11.956922,11.956922,1.776357e-15,11.956922,8.881784e-16,0.000000,1.000000,...,0.000000,225.0,0.214286,0.071429,0.714286,0.000000,False,0.020806,0.005201,-0.03
1,"[[0.91759484 4.49434244 8.04969967] Sr, [-0.68...",mp-1041474.cif,46,5.276693,9.964681,4.687988e+00,8.479494,1.262424e+00,0.093431,1.151553,...,98.972222,12.0,0.380952,0.444444,0.174603,0.000000,False,0.787757,0.171452,2.45
2,"[[4.00319498 2.27224666 2.40977645] Ti, [1.334...",mp-1041486.cif,13,5.876335,12.841077,6.964743e+00,10.411895,2.452624e+00,0.110333,1.116329,...,81.454545,12.0,0.239130,0.347826,0.108696,0.304348,True,0.594445,0.075305,1.63
3,"[[4.03936968 2.14745679 2.56291005] Mo, [1.346...",mp-1041534.cif,13,5.844004,12.841534,6.997530e+00,10.340822,2.402959e+00,0.109013,1.114006,...,86.082645,12.0,0.223404,0.340426,0.138298,0.297872,True,0.336084,0.055831,1.79
4,"[[0.98697611 4.52965491 7.4374028 ] Sr, [-0.70...",mp-1041550.cif,46,5.739410,9.779969,4.040559e+00,8.625092,1.039581e+00,0.083688,1.124143,...,100.527778,12.0,0.347826,0.405797,0.246377,0.000000,False,0.787757,0.161617,3.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,"[[-1.74107977 7.03647892 2.46224834] Ti, [ 5...",mvc-4059.cif,230,4.791369,9.457338,4.665969e+00,8.436354,1.361205e+00,0.083187,1.062393,...,88.222222,12.0,0.375000,0.562500,0.062500,0.000000,True,0.594445,0.116633,1.09
145,"[[1.74759748 1.00897581 7.4144279 ] Si, [ 6.99...",mvc-4354.cif,230,4.768843,9.478858,4.710015e+00,8.447028,1.375650e+00,0.083646,1.062737,...,96.000000,12.0,0.250000,0.375000,0.083333,0.291667,True,0.447278,0.079232,-0.1
146,"[[1.73998689 1.00458183 7.38213889] Si, [ 6.95...",mvc-4519.cif,230,4.796627,9.442975,4.646348e+00,8.427611,1.353711e+00,0.082631,1.061975,...,95.111111,12.0,0.315789,0.473684,0.210526,0.000000,False,0.447278,0.098923,4.08
147,"[[ 0.22280016 1.27259176 -0.23006298] Re, [3....",mvc-637.cif,2,5.578156,11.752605,6.174449e+00,9.722297,2.000784e+00,0.114281,1.130567,...,81.454545,12.0,0.201835,0.293578,0.119266,0.385321,True,0.447278,0.063869,1.43


In [33]:
df_Al['spcacegroup_number']=''

In [34]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer as sa

In [35]:
for index, rows in df_Al.iterrows():
    s=df_Al['structure'][index]
    snumber=sa(s).get_space_group_number()
    df_Al['spcacegroup_number'][index]=snumber

In [36]:
%%time
df_Al = featurizer.featurize_dataframe(df_Al, col_id = "structure")


Wall time: 4min 13s


In [56]:
df_Al.to_csv('df_Al_SVR.csv', index=False)

In [39]:
#y = df_Al ['average_voltage'].values
excluded = ['structure', 'compound possible', 'file_name', 'voltage']
X = df_Al.drop(excluded, axis=1)

In [62]:
y=df_Al['voltage'].values

In [41]:
X_transformed = scaler.transform(X)
#X_test_transformed = scaler.transform (X_test)

In [42]:
X_test_transformed

array([[0.02620087, 0.02833225, 0.29270121, ..., 0.        , 0.85843436,
        0.68511551],
       [0.00436681, 0.02615566, 0.43635825, ..., 0.        , 0.49938331,
        0.36040221],
       [0.31877729, 0.20672927, 0.10479416, ..., 0.        , 0.84336249,
        0.69103445],
       ...,
       [0.05676856, 0.24086644, 0.49515912, ..., 0.        , 0.80797589,
        0.46197076],
       [0.01310044, 0.04774626, 0.44125251, ..., 0.        , 0.85843436,
        0.49638355],
       [0.05676856, 0.03458455, 0.46741385, ..., 0.        , 0.84336249,
        0.62029465]])

In [75]:
X_transformed

array([[ 0.72052402,  0.25720968,  0.43486751, ...,  0.        ,
         0.32975693,  0.3101292 ],
       [ 0.72052402,  0.02627807, -0.23825631, ...,  0.        ,
        -0.04238418, -0.04451269],
       [ 0.6419214 ,  0.51941111,  0.35467244, ...,  0.        ,
         0.00662337,  0.00695599],
       ...,
       [ 0.84279476,  0.25720968,  0.11127028, ...,  0.        ,
         0.17425782,  0.16194214],
       [ 0.26637555,  0.16424746,  0.44650012, ...,  0.        ,
         0.59277853,  0.51137085],
       [ 0.71179039,  0.47507579,  0.43662865, ...,  0.        ,
         0.17425782,  0.16194214]])

In [95]:
y_KRR_pred=KRR.predict(X_transformed)

In [43]:
y_SVR_pred=svr.predict(X_transformed)

In [44]:
y_SVR_pred

array([3.12986786, 1.85959606, 2.57768617, 3.38854763, 2.45513501,
       3.09337964, 3.21384543, 3.61548874, 2.54555868, 3.55742263,
       3.43813962, 2.32975714, 2.71174337, 3.51871568, 4.34609862,
       4.54093802, 1.79081933, 3.72489444, 2.42774902, 4.02447489,
       4.18555   , 3.46122827, 3.43402141, 4.43245342, 4.55036989,
       4.07527556, 1.45878   , 2.95580507, 2.90587261, 2.65263489,
       2.09610641, 2.51733763, 2.42649081, 3.65551571, 2.72836899,
       3.61464938, 2.53751509, 2.5550215 , 2.84420746, 2.66856078,
       1.87479112, 2.18428685, 1.8414565 , 1.76812189, 2.98287585,
       2.76746551, 2.28449522, 2.18207118, 2.17364029, 1.94491371,
       2.41228155, 3.13607273, 2.95648277, 3.03202627, 3.13630003,
       3.13595071, 3.13630003, 2.48053547, 3.05968824, 3.80716799,
       4.58401429, 3.35332076, 2.76236545, 4.94293472, 4.59303932,
       4.50998928, 5.41347907, 3.13610667, 2.96368787, 3.06550392,
       3.13627082, 2.89869342, 2.41256617, 1.94084515, 3.31166

In [66]:
print('test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, 
                                                          y_pred=y_SVR_pred)))
print('test MAE = %.3f' % mean_absolute_error(y_true=y, y_pred=y_SVR_pred))

test RMSE = 1.831
test MAE = 1.534


In [80]:
df_Al

,structure,file_name,spcacegroup_number,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean CN_VoronoiNN,avg_dev CN_VoronoiNN,mean absolute deviation in relative bond length,max relative bond length,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,compound possible,max ionic char,avg ionic char
0,"[[0. 0. 0.] Cu, [1.32833465 0.80296168 5.00398...",CuCl2_mp-1238872_primitive.cif,166,6.000000,11.667937,5.667937e+00,9.778625,2.519083e+00,1.323607e-01,1.099271,...,117.666667,71.555556,64.0,0.200000,0.400000,0.400000,0.0,True,0.327599,0.072800
1,"[[0.57270737 0.35081164 1.89467894] C, [2.8635...",C_mp-169_primitive.cif,166,4.191617,4.191617,2.740919e-12,4.191617,1.370459e-12,2.269296e-13,1.000000,...,194.000000,0.000000,194.0,0.500000,0.500000,0.000000,0.0,True,0.000000,0.000000
2,"[[3.51153342 2.63077745 1.23395558] Mo, [2.598...",Mo3S4_mp-2164_primitive.cif,148,8.053251,10.777218,2.723967e+00,9.604358,1.329520e+00,9.769241e-02,1.136235,...,138.142857,77.877551,70.0,0.261905,0.380952,0.357143,0.0,False,0.043142,0.010565
3,"[[ 4.59337356 2.73964498 20.63148651] Mo, [1....",MoS2_mp-1434_primitive.cif,160,8.708641,12.037374,3.328733e+00,10.927788,1.479432e+00,9.704861e-02,1.072789,...,123.000000,70.666667,70.0,0.277778,0.444444,0.277778,0.0,True,0.043142,0.009587
4,"[[2.04068574 0.99013065 3.10314028] Ni, [3.091...",Ni3S2_mp-362_primitive.cif,155,7.437263,7.970261,5.329975e-01,7.757062,2.558388e-01,4.239864e-03,1.005300,...,163.000000,74.400000,225.0,0.238095,0.190476,0.571429,0.0,False,0.106157,0.025478
5,"[[1.44650159 0.83513808 3.38501344] V, [1.1102...",V2C_mp-1008632_primitive.cif,164,6.000000,9.861925,3.861925e+00,8.574617,1.716411e+00,8.969792e-02,1.067273,...,217.333333,15.555556,229.0,0.428571,0.142857,0.428571,0.0,False,0.190712,0.042380
6,"[[1.47281892 0.8503324 1.08878897] V, [1.1102...",V2C_mp-1094023_primitive.cif,194,6.000000,8.073774,2.073774e+00,7.382516,9.216775e-01,8.671137e-02,1.065034,...,217.333333,15.555556,229.0,0.428571,0.142857,0.428571,0.0,False,0.190712,0.042380
7,"[[1.24650219 2.89935855 6.98050353] V, [0.5618...",V2O5_mp-25280_primitive.cif,62,5.272030,11.797139,6.525109e+00,9.421101,2.370898e+00,1.368482e-01,1.195347,...,74.000000,88.571429,12.0,0.350000,0.500000,0.150000,0.0,True,0.559139,0.114110
8,"[[0. 0. 0.] V, [1.67690291 0.96816035 5.424875...",VSe2_mp-694_primitive.cif,164,7.706069,11.687498,3.981428e+00,10.360355,1.769524e+00,1.000731e-01,1.075055,...,85.666667,95.555556,14.0,0.162162,0.216216,0.621622,0.0,False,0.190712,0.042380
